In [2]:
# import packages
import numpy as np
import random

In [3]:
# fair share queueing
client_names = ["Alice", "Bob", "Carl"] # store names of clients
client_projects = {
    "Alice": np.array([]),
    "Bob": np.array([0.3, 0.1]),
    "Carl": np.array([0.8, 0.2, 0.4]),
}
shares_used = np.array([0.2, 0.5, 0.3]) # store amount of shares used by client

In [4]:
# fair share queueing
def single_fair_share_queuing(client_names, client_projects, shares_used):
    if len(shares_used) == 0:
        return None, None
    
    sorted_shares_used = np.argsort(shares_used)
    print(sorted_shares_used)

    for i in range(len(sorted_shares_used)):
        # get the client with the next lowest use time
        client_name = client_names[sorted_shares_used[i]]
        # get the list of jobs for this client
        client_jobs = client_projects.get(client_name)
        
        # make sure client has jobs requested
        if len(client_jobs) == 0: 
            if i == len(sorted_shares_used) - 1:
                return None, None
            else:
                continue
        else:
            return client_name, client_jobs[np.argsort(client_jobs)[0]]

In [5]:
# fair share queueing
client_name, job = single_fair_share_queuing(client_names, client_projects, shares_used)

print(client_name)
print(job)

[0 2 1]
Carl
0.2


# Multiple Resources
We now seek to expand a fair share queuing system to multiple resources. 

We consider two cases: one, where the resources are pooled and, two, where the resources are broken into sections with upper limits. 

## Continuous Resources
For the purposes of illustration (and my sanity) we consider what plausible queuing algorithms for some set of continuous resources may look like. In reality, a single quantum processor may at most run one job at a time. 

In [6]:
# pooled resources
# TODO: randomize this
client_names = ["Alice", "Bob", "Carl"] # store names of clients
client_projects_and_times = {
    "Alice": (np.array([.6]), np.array([10])),
    "Bob": (np.array([0.3, 0.1]), np.array([50, 20])),
    "Carl": (np.array([0.8, 0.2, 0.4]), np.array([90, 40, 10])),
}
RESOURCE_LIMIT = 1.0
# shares_used = np.array([0.2, 0.5, 0.3])

In [7]:
def is_valid(clients):
    for client in clients:
        if len(clients.get(client)[0]) != len(clients.get(client)[1]):
            return False
    return True

def compile_list(clients):
    if not is_valid(clients):
        return None
    
    out = []
    for client in clients:
        resources = clients.get(client)[0]
        times = clients.get(client)[1]
        
        for i in range(len(resources)):
            out.append((resources[i], times[i], client))
    print(out)
    return out

In [8]:
client_job_list = compile_list(client_projects_and_times)

# randomize the order in which these jobs are run
scrambled_client_job_list = random.shuffle(client_job_list)

print(client_job_list)
print(scrambled_client_job_list)

[(0.6, 10, 'Alice'), (0.3, 50, 'Bob'), (0.1, 20, 'Bob'), (0.8, 90, 'Carl'), (0.2, 40, 'Carl'), (0.4, 10, 'Carl')]
[(0.2, 40, 'Carl'), (0.4, 10, 'Carl'), (0.6, 10, 'Alice'), (0.8, 90, 'Carl'), (0.3, 50, 'Bob'), (0.1, 20, 'Bob')]
None


In [9]:
def pack_schedule(jobs):
    out = [] # stores (job, start_time, end_time)

    curr_resources = 0
    start_time = 0
    last_time = 0
    for job in jobs:
        if job[0] > RESOURCE_LIMIT:
            print("No valid placement for job: " + job)
            continue
        
        if job[0] + curr_resources > RESOURCE_LIMIT: 
            start_time = last_time
            last_time = start_time + job[1]
            end_time = last_time
            curr_resources = job[0]
        else:
            if start_time + job[1] > last_time:
                last_time = start_time + job[1]
            end_time = start_time + job[1]
            curr_resources += job[0]

        out.append(job, start_time, end_time)
        
    return out

In [10]:
# list queuing with no fairness metric
# First Come First Serve (FIFO)
def multiple_continuous_pack_queuing_FIFO(jobs):
    out = pack_schedule(jobs)
    return out

In [11]:
def order_by_time(jobs):
    time_list = [] 
    for job in jobs:
        time_list.append(job[1])

    sorted_time_indices = np.argsort(time_list)

    out = []

    for i in sorted_time_indices:
        out.append(jobs[i])

    return out

# Least Time First (LTF)
def multiple_continuous_pack_queuing_LTF(jobs):
    # reorder jobs in accordance to time
    jobs = order_by_time(jobs)
    out = pack_schedule(jobs)
            
    return out

In [12]:
def order_by_resources(jobs):
    resource_list = []
    for job in jobs:
        resource_list.append(job[0])

    # sort the resource list
    sorted_resource_indices = np.argsort(resource_list)

    out = []

    for i in sorted_resource_indices:
        out.append(jobs[i])

    return out

# Least Resource First (LRF)
def multiple_continuous_pack_queuing_LRF(jobs):
    # reorder jobs in accordance to time
    jobs = order_by_resources(jobs)
    out = pack_schedule(jobs)
            
    return out